### Simple neural network implementation in Numpy

In [28]:
import numpy as np

N = 64 # batch_size
input_dimensions = 500
hidden_neurons = 100
output_dimensions = 10

#Creating random input data
x = np.random.randn(N, input_dimensions)
y = np.random.randn(N, output_dimensions)

#Intializing random weights
w1 = np.random.randn(input_dimensions, hidden_neurons)
w2 = np.random.randn(hidden_neurons, output_dimensions)

learning_rate = 0.000001

for epoch in range(1000):
    #Forward pass to compute y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_predict = h_relu.dot(w2)
    
    
    #Compute loss (mean squared error)
    loss = np.square(y_predict - y).sum()
    print("epoch: ",epoch, " loss: ",loss)
    
    #Backprop to compute weights and loss
    #These can be calculated on the loss function for simple networks. But it gets really difficult to calculate thse
    #for networds with more than 4 layers or networks with complex structure
    grad_y_predict = 2.0 * (y_predict - y)
    grad_w2 = h_relu.T.dot(grad_y_predict)
    grad_h_relu = grad_y_predict.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)
    
    #Updating weights
    w1 = w1 - learning_rate * grad_w1
    w2 = w2 - learning_rate * grad_w2

epoch:  0  loss:  15803048.3567
epoch:  1  loss:  8157447.51707
epoch:  2  loss:  6457810.76282
epoch:  3  loss:  5259611.54489
epoch:  4  loss:  4345295.95371
epoch:  5  loss:  3635261.14597
epoch:  6  loss:  3074339.70489
epoch:  7  loss:  2623764.50181
epoch:  8  loss:  2258804.39031
epoch:  9  loss:  1959057.73364
epoch:  10  loss:  1710256.03485
epoch:  11  loss:  1501477.47317
epoch:  12  loss:  1324879.33634
epoch:  13  loss:  1174357.93785
epoch:  14  loss:  1045382.9616
epoch:  15  loss:  934435.996764
epoch:  16  loss:  838414.607375
epoch:  17  loss:  754936.7251
epoch:  18  loss:  681835.648734
epoch:  19  loss:  617570.840422
epoch:  20  loss:  560873.81727
epoch:  21  loss:  510648.05763
epoch:  22  loss:  465970.956594
epoch:  23  loss:  426158.187938
epoch:  24  loss:  390511.824711
epoch:  25  loss:  358530.261098
epoch:  26  loss:  329734.413795
epoch:  27  loss:  303764.395163
epoch:  28  loss:  280283.914235
epoch:  29  loss:  258999.976513
epoch:  30  loss:  239749

epoch:  272  loss:  26.2619008297
epoch:  273  loss:  25.524020353
epoch:  274  loss:  24.8089153878
epoch:  275  loss:  24.1129182301
epoch:  276  loss:  23.4362749086
epoch:  277  loss:  22.779445805
epoch:  278  loss:  22.1408689202
epoch:  279  loss:  21.5213959497
epoch:  280  loss:  20.9205624462
epoch:  281  loss:  20.3350956156
epoch:  282  loss:  19.7665560776
epoch:  283  loss:  19.2138497833
epoch:  284  loss:  18.6771280987
epoch:  285  loss:  18.1561354916
epoch:  286  loss:  17.6496282888
epoch:  287  loss:  17.1579051013
epoch:  288  loss:  16.6801400716
epoch:  289  loss:  16.2152821689
epoch:  290  loss:  15.7637357087
epoch:  291  loss:  15.325268978
epoch:  292  loss:  14.898973378
epoch:  293  loss:  14.4851720359
epoch:  294  loss:  14.0825089475
epoch:  295  loss:  13.6913039479
epoch:  296  loss:  13.3112143062
epoch:  297  loss:  12.9417313356
epoch:  298  loss:  12.5826720888
epoch:  299  loss:  12.2338147994
epoch:  300  loss:  11.8949831208
epoch:  301  loss:

epoch:  529  loss:  0.0233907973217
epoch:  530  loss:  0.0227750653201
epoch:  531  loss:  0.022176006731
epoch:  532  loss:  0.0215926605478
epoch:  533  loss:  0.021024344634
epoch:  534  loss:  0.0204707028586
epoch:  535  loss:  0.0199320289737
epoch:  536  loss:  0.0194076679143
epoch:  537  loss:  0.018896553472
epoch:  538  loss:  0.0184002930351
epoch:  539  loss:  0.0179169526171
epoch:  540  loss:  0.0174456406392
epoch:  541  loss:  0.0169863724607
epoch:  542  loss:  0.0165396797382
epoch:  543  loss:  0.0161047942963
epoch:  544  loss:  0.0156814523579
epoch:  545  loss:  0.0152695156361
epoch:  546  loss:  0.014868127573
epoch:  547  loss:  0.0144772760294
epoch:  548  loss:  0.0140968256555
epoch:  549  loss:  0.0137263864329
epoch:  550  loss:  0.0133653535679
epoch:  551  loss:  0.0130147510838
epoch:  552  loss:  0.0126732942835
epoch:  553  loss:  0.0123401514903
epoch:  554  loss:  0.0120157547103
epoch:  555  loss:  0.0117003448281
epoch:  556  loss:  0.0113933199

epoch:  803  loss:  1.67940730294e-05
epoch:  804  loss:  1.63592889569e-05
epoch:  805  loss:  1.59352338917e-05
epoch:  806  loss:  1.55222317414e-05
epoch:  807  loss:  1.51208734091e-05
epoch:  808  loss:  1.47299707431e-05
epoch:  809  loss:  1.43482808585e-05
epoch:  810  loss:  1.39766907435e-05
epoch:  811  loss:  1.36146804453e-05
epoch:  812  loss:  1.32624483851e-05
epoch:  813  loss:  1.29186615112e-05
epoch:  814  loss:  1.25840430721e-05
epoch:  815  loss:  1.22580417664e-05
epoch:  816  loss:  1.1941029737e-05
epoch:  817  loss:  1.16318531467e-05
epoch:  818  loss:  1.13305763795e-05
epoch:  819  loss:  1.1037319748e-05
epoch:  820  loss:  1.07515878787e-05
epoch:  821  loss:  1.04738033692e-05
epoch:  822  loss:  1.02027548132e-05
epoch:  823  loss:  9.93870587981e-06
epoch:  824  loss:  9.68154352132e-06
epoch:  825  loss:  9.43089313229e-06
epoch:  826  loss:  9.18658128078e-06
epoch:  827  loss:  8.94874892777e-06
epoch:  828  loss:  8.71730463423e-06
epoch:  829  l

In [32]:
import torch


N = 64 # batch_size
input_dimensions = 500
hidden_neurons = 100
output_dimensions = 10

#Creating random input data
x = torch.randn(N, input_dimensions).type(torch.FloatTensor)
y = torch.randn(N, output_dimensions).type(torch.FloatTensor)

#Intializing random weights
w1 = torch.randn(input_dimensions, hidden_neurons).type(torch.FloatTensor)
w2 = torch.randn(hidden_neurons, output_dimensions).type(torch.FloatTensor)

learning_rate = 0.000001

for epoch in range(1000):
    #Forward pass to compute y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_predict = h_relu.mm(w2)
    
    
    #Compute loss (mean squared error)
    loss = (y_predict - y).pow(2).sum()
    print("epoch: ",epoch, " loss: ",loss)
    
    #Backprop to compute weights and loss
    #These can be calculated on the loss function for simple networks. But it gets really difficult to calculate thse
    #for networds with more than 4 layers or networks with complex structure
    grad_y_predict = 2.0 * (y_predict - y)
    grad_w2 = h_relu.t().mm(grad_y_predict)
    grad_h_relu = grad_y_predict.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    #Updating weights
    w1 = w1 - learning_rate * grad_w1
    w2 = w2 - learning_rate * grad_w2

epoch:  0  loss:  14165516.352895014
epoch:  1  loss:  8657232.725245625
epoch:  2  loss:  6786233.04906757
epoch:  3  loss:  5451484.811289549
epoch:  4  loss:  4451314.694844216
epoch:  5  loss:  3684584.4369721934
epoch:  6  loss:  3085171.7063257694
epoch:  7  loss:  2609236.7763623046
epoch:  8  loss:  2226718.6621317985
epoch:  9  loss:  1914726.552116792
epoch:  10  loss:  1657640.6854792736
epoch:  11  loss:  1443583.3972576018
epoch:  12  loss:  1264453.3455534326
epoch:  13  loss:  1113255.9235138015
epoch:  14  loss:  984789.9876688616
epoch:  15  loss:  875018.5818134199
epoch:  16  loss:  780620.5419665719
epoch:  17  loss:  699054.0385428262
epoch:  18  loss:  628161.2496127386
epoch:  19  loss:  566391.2689181196
epoch:  20  loss:  512336.03517907904
epoch:  21  loss:  464776.78341705323
epoch:  22  loss:  422756.81447576825
epoch:  23  loss:  385484.9046516933
epoch:  24  loss:  352370.62196399644
epoch:  25  loss:  322816.72886903724
epoch:  26  loss:  296369.197729977

epoch:  298  loss:  3.163021491116723
epoch:  299  loss:  3.061952509546913
epoch:  300  loss:  2.9642870372585
epoch:  301  loss:  2.8694951069082535
epoch:  302  loss:  2.777930305892943
epoch:  303  loss:  2.6893879180619464
epoch:  304  loss:  2.603726171743472
epoch:  305  loss:  2.520819557886067
epoch:  306  loss:  2.4406451930427666
epoch:  307  loss:  2.3630804836952493
epoch:  308  loss:  2.2879340512725745
epoch:  309  loss:  2.2150576312809065
epoch:  310  loss:  2.144812883904387
epoch:  311  loss:  2.0768291124359175
epoch:  312  loss:  2.0111134436682585
epoch:  313  loss:  1.947272166893725
epoch:  314  loss:  1.8854996546316523
epoch:  315  loss:  1.8257443262541964
epoch:  316  loss:  1.7681195148154671
epoch:  317  loss:  1.7121837873244914
epoch:  318  loss:  1.6581443539478506
epoch:  319  loss:  1.6055752957550915
epoch:  320  loss:  1.5550232054729272
epoch:  321  loss:  1.5059702348960473
epoch:  322  loss:  1.4584394487227712
epoch:  323  loss:  1.4124535401500

epoch:  754  loss:  7.344858755267171e-05
epoch:  755  loss:  7.285658932202482e-05
epoch:  756  loss:  7.259238865845591e-05
epoch:  757  loss:  7.19279820374577e-05
epoch:  758  loss:  7.136342829868608e-05
epoch:  759  loss:  7.084481803139309e-05
epoch:  760  loss:  7.026103597757016e-05
epoch:  761  loss:  6.96439992540418e-05
epoch:  762  loss:  6.883497277641176e-05
epoch:  763  loss:  6.834644654243188e-05
epoch:  764  loss:  6.786970590010738e-05
epoch:  765  loss:  6.727945260939783e-05
epoch:  766  loss:  6.674772857399869e-05
epoch:  767  loss:  6.641058744712158e-05
epoch:  768  loss:  6.592622596550413e-05
epoch:  769  loss:  6.546716158627175e-05
epoch:  770  loss:  6.481701021757158e-05
epoch:  771  loss:  6.409925938632988e-05
epoch:  772  loss:  6.3610338857642e-05
epoch:  773  loss:  6.311139396421306e-05
epoch:  774  loss:  6.263707556940223e-05
epoch:  775  loss:  6.240205358315487e-05
epoch:  776  loss:  6.188751414404337e-05
epoch:  777  loss:  6.11208778809394e-

epoch:  960  loss:  2.277579842870081e-05
epoch:  961  loss:  2.270022815584888e-05
epoch:  962  loss:  2.2600440389392712e-05
epoch:  963  loss:  2.25348189987519e-05
epoch:  964  loss:  2.2403888965766283e-05
epoch:  965  loss:  2.2331236920917347e-05
epoch:  966  loss:  2.2216012071432645e-05
epoch:  967  loss:  2.214797161581919e-05
epoch:  968  loss:  2.203958299125397e-05
epoch:  969  loss:  2.196039002018424e-05
epoch:  970  loss:  2.1843446407151385e-05
epoch:  971  loss:  2.1810121121120657e-05
epoch:  972  loss:  2.1681060277476544e-05
epoch:  973  loss:  2.166380847669e-05
epoch:  974  loss:  2.162186179175199e-05
epoch:  975  loss:  2.1531209957698777e-05
epoch:  976  loss:  2.1525845398124438e-05
epoch:  977  loss:  2.1476855925576382e-05
epoch:  978  loss:  2.1324609375903553e-05
epoch:  979  loss:  2.1283293188251867e-05
epoch:  980  loss:  2.1184064222563512e-05
epoch:  981  loss:  2.1139722600961586e-05
epoch:  982  loss:  2.1084107518557632e-05
epoch:  983  loss:  2.0